In [1]:
import os
from langchain.embeddings.groq import GroqEmbeddings
from dotenv import load_dotenv

load_dotenv()

# Load your Groq API key from .env
api_key = os.getenv("GROQ_API_KEY")

# Initialize Groq Embeddings
embeddings_model = GroqEmbeddings(api_key=api_key)

# Example short sentence
sentence = "Tata Consultancy Services is a leading IT company in India."

# Generate embeddings
vector = embeddings_model.embed_query(sentence)

print("Embedding vector length:", len(vector))
print("First 10 dimensions:", vector[:10])


ModuleNotFoundError: No module named 'langchain.embeddings.groq'

In [2]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
texts = ["Infosys is a multinational IT company."]
vectorstore = FAISS.from_texts(texts, embeddings)


C:\Users\Student\AppData\Local\Temp\ipykernel_18544\4077586286.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\Student\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
vectorstore

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings

# Initialize the embedding model
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Example text
text = "Infosys is a multinational IT company."

# Generate embedding
vector = embeddings_model.embed_query(text)

# Print embedding
print("Embedding vector length:", len(vector))
print("First 10 dimensions:", vector[:10])  # print first 10 numbers for readability


Embedding vector length: 384
First 10 dimensions: [-0.030184494331479073, -0.02259017527103424, 0.00886974111199379, -0.05324116349220276, 0.05300078168511391, -0.14952747523784637, 0.07439170777797699, 0.006610870361328125, -0.009535393677651882, -0.03792954608798027]


In [10]:
import os
import pandas as pd
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# -----------------------------
# Load environment variables
# -----------------------------
load_dotenv()
api_key = os.getenv("GROQ_API_KEY")

# -----------------------------
# Load saved vector store
# -----------------------------
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.load_local("faiss_index", embeddings_model, allow_dangerous_deserialization=True)
print("Vector store loaded!")

# -----------------------------
# Initialize ChatGroq LLM
# -----------------------------
llm = ChatGroq(
    api_key=api_key,
    model_name="llama-3.3-70b-versatile",
    temperature=0.7
)

# -----------------------------
# Define LangChain ChatPromptTemplate
# -----------------------------
system_template = """You are a helpful assistant. 
You can optionally call a tool to retrieve company stock symbols if needed.
Use the tool results only if relevant, otherwise answer normally."""
system_msg_prompt = SystemMessagePromptTemplate.from_template(system_template)

human_template = """User Question:
{user_query}

Tool Output (use only if relevant):
{retrieved_info}

Answer:"""
human_msg_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_msg_prompt, human_msg_prompt])

# -----------------------------
# Define the "tool"
# -----------------------------
def symbol_lookup_tool(query_str, top_k=2):
    """
    Returns top_k most similar company entries from the vector store.
    """
    docs = vectorstore.similarity_search(query_str, k=top_k)
    results = [doc.page_content for doc in docs]
    return results

# -----------------------------
# Chat loop
# -----------------------------
print("ChatGroq Symbol Bot ready! Type 'exit' to quit.")

while True:
    user_query = input("You: ")
    if user_query.lower() in ["exit", "quit"]:
        break

    # Step 1: Ask the model whether to use the tool and what string to query
    tool_decision_prompt = f"""
You are a chatbot. The user asked: "{user_query}".
Decide if you need to call the 'symbol_lookup_tool' to find company symbols.
If yes, generate a short string to query the tool (do not just repeat the user input).
If no, just respond 'NO_TOOL'.
Return only one line: either 'NO_TOOL' or the query string to pass to the tool.
"""
    decision_response = llm.invoke([
        ("system", "You are a helpful assistant."),
        ("human", tool_decision_prompt)
    ])
    tool_query = decision_response.content.strip()
    
    # Step 2: Call the tool if needed
    if tool_query != "NO_TOOL":
        tool_results = symbol_lookup_tool(tool_query, top_k=2)
        retrieved_info = "\n".join(tool_results)
        print(f"[DEBUG] Model chose to call tool with query: '{tool_query}'")
        print(f"[DEBUG] Tool returned top 2 results:\n{retrieved_info}")
    else:
        retrieved_info = "No tool call needed."
        print(f"[DEBUG] Model chose NOT to call the tool.")

    # Step 3: Format messages using ChatPromptTemplate
    messages = chat_prompt.format_messages(user_query=user_query, retrieved_info=retrieved_info)

    # Step 4: Get final response from model
    response = llm.invoke(messages)
    
    print("Bot:", response.content)


Vector store loaded!
ChatGroq Symbol Bot ready! Type 'exit' to quit.
[DEBUG] Model chose NOT to call the tool.
Bot: Hello. How can I assist you today?
[DEBUG] Model chose NOT to call the tool.
Bot: The symbol of the Nifty index is ^NSEI or NIFTY. However, it's often represented by its benchmark index symbol, which is NSE:NIFTY or .NSEI.
[DEBUG] Model chose to call tool with query: 'nifty index symbol'
[DEBUG] Tool returned top 2 results:
Company: NIFTYHOUSING-INDEX, Symbol: NSE:NIFTYHOUSING-INDEX
Company: NIFTYINFRA-INDEX, Symbol: NSE:NIFTYINFRA-INDEX
Bot: The Fyers symbol for Nifty is "NIFTY".
[DEBUG] Model chose to call tool with query: 'nifty 50 company symbols'
[DEBUG] Tool returned top 2 results:
Company: NIFTY ALPHA 50-INDEX, Symbol: NSE:NIFTYALPHA50-INDEX
Company: NIFTY50SHARIAH-INDEX, Symbol: NSE:NIFTY50SHARIAH-INDEX
Bot: The Nifty 50 is a stock market index composed of 50 of the largest and most liquid Indian companies. It is one of the main stock indices used to measure the p